In [59]:
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm
from joblib import Parallel, delayed
from math import floor, ceil
import os

In [2]:
DATA_DIR = '..\\data\\raw'
INTERIM_DIR = '..\\data\\interim'

### Initial processing of files

In [3]:
df = pd.read_csv(os.path.join(DATA_DIR, 'title.basics.tsv'), sep='\t')
df.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [4]:
# Listing all possible titleTypes

df['titleType'].unique()

array(['short', 'movie', 'tvShort', 'tvMovie', 'tvSeries', 'tvEpisode',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'episode'],
      dtype=object)

## Strategy

- Framing the problem as predicting movie ratings for 'movie' and 'tvMovie' 
- Convert the 'startYear' field to numeric values and drop the 'endYear' field as it is irrelevant for movies
- Reducing the scope of the problem by filtering movies to the ones that were released after 1950 

#### Treating missing values for startYear:

- Currently, we are setting the startYear as 0 for missing values. However, a better approach would be to impute the year of the movie from other tables. For eg. we have the data on which actors/directors/crew worked on the movie from title.crew and title.principals files. We could collect the crew names and find them in name.basics file. Then we could compute a time when all of these people were alive (a time between [max of their birthYear + some constant] and [min of their deathYear]) and choose a year from that time.  

In [5]:
del df['endYear']
df['startYear'] = df['startYear'].apply(lambda x: int(x) if (x!='\\N') else 0)

movies = df.loc[((df['titleType']=='movie') | (df['titleType']=='tvMovie')) & (df['startYear']>1950)]
movies.reset_index(drop=True, inplace=True)
movies.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0011216,movie,Spanish Fiesta,La fête espagnole,0,2019,67,Drama
1,tt0011801,movie,Tötet nicht mehr,Tötet nicht mehr,0,2019,\N,"Action,Crime"
2,tt0015414,movie,La tierra de los toros,La tierra de los toros,0,2000,60,\N
3,tt0015724,movie,Dama de noche,Dama de noche,0,1993,102,"Drama,Mystery,Romance"
4,tt0016906,movie,Frivolinas,Frivolinas,0,2014,80,"Comedy,Musical"


In [6]:
len(movies)

557381

### Dealing with Null Values

In [7]:
def process_null_values(df):
    '''
    Inputs a dataframe and handles the null values.
    Since data uses a non-standard identifier ('\\N') for null,
    we convert all these values element-wise to np.NaN for pandas processing
    '''
    return df.applymap(lambda x: np.NaN if (x=='\\N') else x)

In [8]:
## Dataset has '\\N' in place of NaN therefore converting it to NaN
df = process_null_values(df)
movies = process_null_values(movies)
movies

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0011216,movie,Spanish Fiesta,La fête espagnole,0,2019,67,Drama
1,tt0011801,movie,Tötet nicht mehr,Tötet nicht mehr,0,2019,NaN,"Action,Crime"
2,tt0015414,movie,La tierra de los toros,La tierra de los toros,0,2000,60,NaN
3,tt0015724,movie,Dama de noche,Dama de noche,0,1993,102,"Drama,Mystery,Romance"
4,tt0016906,movie,Frivolinas,Frivolinas,0,2014,80,"Comedy,Musical"
...,...,...,...,...,...,...,...,...
557376,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007,100,Documentary
557377,tt9916692,tvMovie,Teatroteka: Czlowiek bez twarzy,Teatroteka: Czlowiek bez twarzy,0,2015,66,Drama
557378,tt9916706,movie,Dankyavar Danka,Dankyavar Danka,0,2013,NaN,Comedy
557379,tt9916730,movie,6 Gunn,6 Gunn,0,2017,116,NaN


In [9]:
movies.isnull().sum()

tconst                 0
titleType              0
primaryTitle           0
originalTitle          0
isAdult                0
startYear              0
runtimeMinutes    147824
genres             57764
dtype: int64

#### Treating missing values for genres:

- For treating missing values of genres, the following approaches come to mind:  
  
  
  - **Calculating genres from crew**: (Implementation later in the notebook)
    1. Using the crew and principals file, we could get the people associated (nconsts) with the title
    2. Using the 'knownForTitles' field in the name.basics file, we could get a list of all titles (tconsts) these people are known for.
    3. Using the title.basics file, we now can get the genres associated with the above titles and generate a dictionary of counts of genres that appear in these titles. 
    4. We then take the genres with the top 3 counts (since genres can have at most 3 strings) in that dictionary and associate it with the title.
    
    **NOTE:** We use this approach only when the genres field is empty, we do not do it for records where there are less than 3 strings as it is not logical to do so in my opinion. 
  
      
  - **Using Word embeddings with text of reviews**:  
    1. If we had the text reviews of the movies available, we could use a word embedding to associate the reviews with the available genres. In particular, we could use the words occuring in the reviews and calculate their similarity distance (cosine distance) with each of the genres that we find in our dataset. We thus get a dictionary of genres and their similarity with the reviews. We could again take the genres with the top 3 values in this vector and use those genres. 

In [10]:
# Collecting all possible movie genres by splitting the genres field by commas and adding them to a set

genres = set()
movies['genres'].apply(lambda x: genres.update(x.split(',')) if isinstance(x, str) else False)
genres

{'Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western'}

In [11]:
len(genres)

28

#### Replacing NaN values in runtimeMinutes by mean of the column

In [12]:
mean_runtime = int(movies['runtimeMinutes'].dropna().astype(int).mean())

movies['runtimeMinutes'] = movies['runtimeMinutes'].apply(lambda x: mean_runtime if (pd.isna(x)) else x)

### Processing the names.basics, title.crew, title.principals files for handling the genre missing values

In [13]:
names = pd.read_csv(os.path.join(DATA_DIR, 'name.basics.tsv'), sep='\t')
names = process_null_values(names)
names

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0050419,tt0031983,tt0053137,tt0072308"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0038355,tt0117057,tt0071877"
2,nm0000003,Brigitte Bardot,1934,NaN,"actress,soundtrack,music_department","tt0057345,tt0054452,tt0049189,tt0056404"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0077975,tt0078723,tt0072562,tt0080455"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0060827,tt0050976,tt0069467"
...,...,...,...,...,...,...
10852297,nm9993714,Romeo del Rosario,NaN,NaN,"animation_department,art_department",tt2455546
10852298,nm9993716,Essias Loberg,NaN,NaN,NaN,NaN
10852299,nm9993717,Harikrishnan Rajan,NaN,NaN,cinematographer,tt8736744
10852300,nm9993718,Aayush Nair,NaN,NaN,cinematographer,NaN


In [14]:
names.isnull().sum()

nconst                      0
primaryName                 0
birthYear            10329355
deathYear            10664496
primaryProfession     2294306
knownForTitles        1999277
dtype: int64

In [15]:
# Getting the distribution of the counts of knownforTitles across the dataset

names['knownForTitles'].apply(lambda x: len(x.split(',')) if (not pd.isna(x)) else 0).describe()

count    1.085230e+07
mean     1.562484e+00
std      1.341015e+00
min      0.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      6.000000e+00
Name: knownForTitles, dtype: float64

In [16]:
title_crew = pd.read_csv(os.path.join(DATA_DIR, 'title.crew.tsv'), sep='\t')
title_crew = process_null_values(title_crew)
title_crew.head()

,tconst,directors,writers
0,tt0000001,nm0005690,NaN
1,tt0000002,nm0721526,NaN
2,tt0000003,nm0721526,NaN
3,tt0000004,nm0721526,NaN
4,tt0000005,nm0005690,NaN


In [17]:
len(title_crew)

7803001

In [18]:
title_principals = pd.read_csv(os.path.join(DATA_DIR, 'title.principals.tsv'), sep='\t')
title_principals = process_null_values(title_principals)
title_principals.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,NaN,"[""Self""]"
1,tt0000001,2,nm0005690,director,NaN,NaN
2,tt0000001,3,nm0374658,cinematographer,director of photography,NaN
3,tt0000002,1,nm0721526,director,NaN,NaN
4,tt0000002,2,nm1335271,composer,NaN,NaN


In [19]:
len(title_principals)

44119644

#### We do not need all the records from the title.crew and title.principals files, we just need the ones associated with the movies we are considering

In [20]:
# Filtering the crew and principals dataframes to include only relevant titles

# Add the titles we are considering to a set and then filter the dataframes
considered_titles = set()
considered_titles.update(movies['tconst'].values)

title_principals = title_principals[title_principals['tconst'].isin(considered_titles)]
title_crew = title_crew[title_crew['tconst'].isin(considered_titles)]

print("Length of processed title_crew: {}".format(len(title_crew)))
print("Length of processed title_principals: {}".format(len(title_principals)))

Length of processed title_crew: 557381
Length of processed title_principals: 4196118


In [21]:
# Saving files as csvs to Interim data folder

title_crew.to_csv(os.path.join(INTERIM_DIR, 'title.crew.csv'), index=False)
title_principals.to_csv(os.path.join(INTERIM_DIR, 'title.principals.csv'), index=False)

#### Filtering the names.basics dataframe for including only the names associated with considered movies for faster processing

**NOTE**: This filtering is only for saving time in prototyping further in the pipeline as I plan to calculate features for each person. In production, I would retain the entire name list and calculate the features on them as well in case a movie has a crew member who hasnt worked in a movie before (eg. an actor who only did TV series up until now).

In [22]:
considered_name_ids = set()

def split_nameIDs(nconst_str):
    '''
    Split the nconst string by commas and filter out empty strings
    '''
    return [nameID for nameID in nconst_str.split(',') if nameID!='']

# Adding all directors
title_crew['directors'].dropna().apply(lambda x: considered_name_ids.update(split_nameIDs(x)))

# Adding all writers
title_crew['writers'].dropna().apply(lambda x: considered_name_ids.update(split_nameIDs(x)))

# Adding principal crew members
considered_name_ids.update(title_principals['nconst'].dropna().values)

print(len(considered_name_ids))

1434596


In [23]:
names = names[names['nconst'].isin(considered_name_ids)]
names.reset_index(drop=True, inplace=True)
names

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0050419,tt0031983,tt0053137,tt0072308"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0038355,tt0117057,tt0071877"
2,nm0000003,Brigitte Bardot,1934,NaN,"actress,soundtrack,music_department","tt0057345,tt0054452,tt0049189,tt0056404"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0077975,tt0078723,tt0072562,tt0080455"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0060827,tt0050976,tt0069467"
...,...,...,...,...,...,...
1433317,nm9993650,Marcin Balcerak,NaN,NaN,actor,tt8739208
1433318,nm9993680,Christopher-Lawson Palmer,NaN,NaN,actor,"tt10427366,tt10979852,tt8295580"
1433319,nm9993690,David Jewell,NaN,NaN,NaN,tt7888884
1433320,nm9993691,Ursula Gehrmann,NaN,NaN,NaN,tt7888884


In [24]:
names.to_csv(os.path.join(INTERIM_DIR, 'names.basics.csv'))

### Pipeline for processing NaN values in genres

In [25]:
genre_null_index = movies[movies['genres'].isna()].index

In [26]:
movies.iloc[list(genre_null_index)]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
2,tt0015414,movie,La tierra de los toros,La tierra de los toros,0,2000,60,NaN
5,tt0019996,movie,Hongxia,Hongxia,0,2011,94,NaN
18,tt0036574,movie,"January 30, 1945",30. Januar 1945,0,1965,146,NaN
28,tt0039952,movie,Vecchio cinema... che passione!,Vecchio cinema... che passione!,0,1957,86,NaN
83,tt0042443,movie,Facultad de letras,Facultad de letras,0,1952,70,NaN
...,...,...,...,...,...,...,...,...
557318,tt9913320,movie,Seitô seihanzai,Seitô seihanzai,0,1981,71,NaN
557327,tt9913814,movie,Chikan densha: Muremure shanai,Chikan densha: Muremure shanai,0,1981,60,NaN
557328,tt9913834,movie,Hiniku: Nure nawazeme,Hiniku: Nure nawazeme,0,1981,64,NaN
557330,tt9913878,movie,Document porno: Yubi ijime,Document porno: Yubi ijime,0,1981,61,NaN


In [45]:
def get_genres(tconst, debug=False):
    '''
    Given a tconst that has no genre associated with it, 
    return a string with 3 genres comma-separated estimated
    using the top 3 genres its crew is involved in.
    '''
    
    title_crew = pd.read_csv(os.path.join(INTERIM_DIR, 'title.crew.csv'), usecols=['tconst', 'directors', 'writers'])
    title_principals = pd.read_csv(os.path.join(INTERIM_DIR, 'title.principals.csv'), usecols=['tconst', 'nconst'])
    names = pd.read_csv(os.path.join(INTERIM_DIR, 'names.basics.csv'), usecols=['nconst', 'knownForTitles'])
    
    people_involved = set()
    
    # Adding actors associated with the movie to the set
    people_involved.update(title_principals[title_principals['tconst']==tconst]['nconst'].values)
    
    # Adding writers and directors
    crew = title_crew[title_crew['tconst']==tconst]
    
    if not pd.isna(crew['writers'].iloc[0]):
        people_involved.update(split_nameIDs(crew['writers'].iloc[0]))
    
    if not pd.isna(crew['directors'].iloc[0]):
        people_involved.update(split_nameIDs(crew['directors'].iloc[0]))
    
    # Choose the people involved in the names list
    people_involved_df = names[names['nconst'].isin(people_involved)]
    
    if debug:
        print(people_involved_df)
    
    # We just need the titles that they are collectively known for
    people_titles = people_involved_df['knownForTitles'].dropna()
    
    people_titles_set = set()
    people_titles.apply(lambda x: people_titles_set.update(split_nameIDs(x)))
    
    if debug:
        print(people_titles_set)
    
    # Filter these records from title.basics file to get all the genres associated with them
    genres = df[df['tconst'].isin(people_titles_set)]['genres'].dropna()
    genres_list = []
    genres.apply(lambda x: genres_list.extend(split_nameIDs(x)))
    genres = Counter(genres_list)
    
    if debug:
        print(genres)
        print(genres.most_common(3))
    
    genres = [genre for (genre, count) in genres.most_common(3)]
    
    return ','.join(genres)

#### Trying out the get_genres() function

In [46]:
get_genres('tt0015414', debug=True)

           nconst                           knownForTitles
56046   nm0147437  tt0218852,tt0031182,tt0015414,tt0013620
206917  nm0615736  tt0006206,tt2122391,tt0268018,tt0011594
{'tt0268018', 'tt0015414', 'tt0218852', 'tt0013620', 'tt0006206', 'tt2122391', 'tt0031182', 'tt0011594'}
Counter({'Drama': 3, 'Action': 2, 'Crime': 2, '\\N': 2, 'Short': 2, 'Adventure': 1, 'Documentary': 1, 'Romance': 1, 'Mystery': 1})
[('Drama', 3), ('Action', 2), ('Crime', 2)]


'Drama,Action,Crime'

#### Applying the get genres function to the records having no genres associated with them TODO: Parallel implementation using joblib


In [50]:
# Get the indices of the rows with no genres
no_gen_ind = movies.index[movies['genres'].isna()].tolist()

# for each index in this list, run the get_genres function and replace the genres column

for row_index in tqdm(no_gen_ind):
    
    tconst = movies['tconst'].iloc[row_index]
    movies['genres'].iloc[row_index] = get_genres(tconst)

# Check the output for rows that were in the index 
movies[no_gen_ind]

  0%|                                                                                        | 0/57764 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
  0%|▏                                                                         | 115/57764 [23:37<194:45:00, 12.16s/it]

KeyboardInterrupt: 

#### Parallel implementation using joblib

In [60]:
def get_genres_parallel(tconst_list, debug=False):
    
    num_cores = os.cpu_count()
    items_per_core = ceil(len(tconst_list)/num_cores)
    
    list_per_core = [tconst_list[i*items_per_core: (i+1)*items_per_core]
                     for i in range(num_cores)]

    title_crew = pd.read_csv(os.path.join(INTERIM_DIR, 'title.crew.csv'), usecols=['tconst', 'directors', 'writers'])
    title_principals = pd.read_csv(os.path.join(INTERIM_DIR, 'title.principals.csv'), usecols=['tconst', 'nconst'])
    names = pd.read_csv(os.path.join(INTERIM_DIR, 'names.basics.csv'), usecols=['nconst', 'knownForTitles'])
    
    Parallel(n_jobs=-1)(delayed(get_genres(tconst_list, title_crew.copy(), 
                                           title_principals.copy(), names.copy())))

In [ ]:
def get_genres(tconst_list, title_crew, title_principals, names):
    
    for (ind, tconst) in tconst_list:
        
        people_involved = set()
    
        # Adding actors associated with the movie to the set
        people_involved.update(title_principals[title_principals['tconst']==tconst]['nconst'].values)

        # Adding writers and directors
        crew = title_crew[title_crew['tconst']==tconst]

        if not pd.isna(crew['writers'].iloc[0]):
            people_involved.update(split_nameIDs(crew['writers'].iloc[0]))

        if not pd.isna(crew['directors'].iloc[0]):
            people_involved.update(split_nameIDs(crew['directors'].iloc[0]))

        # Choose the people involved in the names list
        people_involved_df = names[names['nconst'].isin(people_involved)]

        if debug:
            print(people_involved_df)

        # We just need the titles that they are collectively known for
        people_titles = people_involved_df['knownForTitles'].dropna()

        people_titles_set = set()
        people_titles.apply(lambda x: people_titles_set.update(split_nameIDs(x)))

        if debug:
            print(people_titles_set)

        # Filter these records from title.basics file to get all the genres associated with them
        genres = df[df['tconst'].isin(people_titles_set)]['genres'].dropna()
        genres_list = []
        genres.apply(lambda x: genres_list.extend(split_nameIDs(x)))
        genres = Counter(genres_list)

        if debug:
            print(genres)
            print(genres.most_common(3))

        genres = [genre for (genre, count) in genres.most_common(3)]